In [1]:
import pandas as pd
import numpy as np

import os
import tarfile

In [3]:
# Unzip NOAA observation dataset

inp_folder = r'd:/Dataset/NOAA_2/'
out_folder = r'D:\Projects\precipitation_AUS\dataset/NOAA/'

entries = os.listdir(inp_folder)

for f in entries:
    print(f)
    inp = inp_folder + f
    out = out_folder + f
    try : os.mkdir( out)
    except: pass
    tar = tarfile.open(inp)
    tar.extractall(out)
    tar.close()

2023.tar.gz
2024.tar.gz


In [3]:

# Find study area CSV files using lat , long

folder_NOAA = r'D:\Projects\precipitation_AUS\dataset/NOAA/'
folder_out = r'D:\Projects\precipitation_AUS\dataset/NOAA/AUS/'

lat_ext = (-40 , -10 ) # extent of latitude
long_ext = (110 , 160 ) # extent of longitudt


try: os.mkdir(folder_out)
except: pass 

entries = os.listdir(folder_NOAA)


for i in range(16 , len(entries)):
    print( entries[i])
    fp_year = folder_NOAA + entries[i]
    csv_names = os.listdir(fp_year)
    for csv in csv_names:
        fp_csv = fp_year + '/' + csv 
        CSV = pd.read_csv(fp_csv)
        if   lat_ext[0] <= CSV['LATITUDE'][0] <= lat_ext[1] and  long_ext[0] <= CSV['LONGITUDE'][0] <= long_ext[1] :
            fp_out = folder_out + entries[i][0:4] + '_'  + CSV['NAME'][0].replace( '\\'  , ' ')  +'.csv'
            CSV.to_csv(fp_out) 
 





2010.tar.gz
2011.tar.gz
2012.tar.gz
2013.tar.gz
2014.tar.gz
2015.tar.gz
2016.tar.gz
2017.tar.gz
2018.tar.gz
2019.tar.gz
2020.tar.gz
2021.tar.gz
2022.tar.gz
2023.tar.gz
2024.tar.gz
AUS


KeyboardInterrupt: 

In [5]:
# Put all CSV dile in a single netcdf file 

from tqdm import tqdm
folder = r'D:\Projects\precipitation_AUS\dataset/NOAA/AUS/'
entries = os.listdir( folder )
N= len(entries)
List = []
for i in tqdm(range( N), desc="Sampling..." , ncols= 100  ,colour='blue'):
    file = entries[i]
    fp = folder + file 
    csv = pd.read_csv(fp)
    List.append(csv)
df = pd.concat(List)
of = r"D:\Projects\precipitation_AUS\dataset/noaa_raw_obs.nc4"
import xarray
xr = xarray.Dataset(df)
xr.to_netcdf(of)

Sampling...: 100%|████████████████████████████████████████████| 15294/15294 [13:18<00:00, 19.16it/s]


In [19]:
# identify number of stations and their names
uniq =  df['NAME'].unique()
print( len(uniq) , 
df.keys())

In [28]:
# exclude non values as 99.99
df_0 = df[ df['PRCP'] != 99.99 ]

In [46]:
# exclude stations with a few observations
uniq =  df_0['NAME'].unique()
N= len(uniq)
for i in tqdm(range( N), desc="Sampling..." , ncols= 100  ,colour='blue'):
    u = uniq[i]
    df_name = df_0[df_0['NAME'] == u ]
    if len(df_name) < 365:
        df_0 = df_0[df_0['NAME'] != u]


In [53]:

# split the date column into three columns
df_0[['YEAR', 'MONTH' , 'DAY' ]] = df_0['DATE'].str.split('-', expand=True)



In [55]:
# save observations 
of = r"D:\Projects\precipitation_AUS\dataset/noaa_obs.nc4"
xr_0 = xarray.Dataset(df_0)
xr_0.to_netcdf(of)

In [64]:
# str to int
df_0['YEAR'] = np.array(df_0["YEAR"] , dtype= int)
df_0['MONTH'] = np.array(df_0["MONTH"] , dtype= int)
df_0['DAY'] = np.array(df_0["DAY"] , dtype= int)

In [66]:
# make a query based on the year 
df_1 = df_0[ df_0['YEAR' ]  >= 2014 ] 
of = r"D:\Projects\precipitation_AUS\dataset/noaa_obs_2014-2024.nc4"
xr_1 = xarray.Dataset(df_1)
xr_1.to_netcdf(of)